### Python modules

In [1]:
import os, requests, json
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from matplotlib.path import Path as mpl_path

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Sistema Comando API Urls

In [63]:
api_root = 'https://api.dados.rio/v2/adm_cor_comando'

pops_url = api_root + '/pops'

pop_url = api_root + '/procedimento_operacional_padrao_orgaos_responsaveis'

open_events_url = api_root + '/ocorrencias_abertas'

event_actions_url = '/ocorrencias_orgaos_responsaveis'

### Sistema Comando API - Clusters Polygons Extension Methods

In [258]:
# Load POPs dataframe
pops_path = 'pops.csv'
pops_df = pd.read_csv(pops_path)
pops_map = pops_df.set_index('id')['titulo'].to_dict()

# Load custers polygons geojson data
polygons_geojson_path = '../Hackaton COR IV - Centro de Operações do RJ/ACELERAÇÃO/Dados/Clusters/polygons_micro.geojson'
polygons_geojson = json.loads(open(polygons_geojson_path, 'r').read())

# Get polygons dict from geojson
polygons = {}
for poly in polygons_geojson['features']:
    if poly['properties']['sublabel'] != -1:
        polygons[poly['properties']['sublabel']] = poly['geometry']['coordinates'][0]

# ---
# Sistema Comando API - Clusters Polygons Extension Methods

def get_open_events():
    open_events_res, i = {'error': ''}, 1
    while 'eventos' not in list(open_events_res.keys()):
        print(f'Api comando open events: Request attempt ({i})'); i += 1
        open_events_res = json.loads(requests.get(open_events_url).text)
    open_events_df =  pd.DataFrame(open_events_res['eventos'])
    open_events_df['pop_titulo'] = open_events_df['pop_id'].map(pops_map) # include event pop title
    return open_events_df

def get_open_events_extended():
    
    # Get open events dataframe
    open_events_df = get_open_events()
    # Get events points as tuple array
    points = np.array(list(map(tuple, open_events_df[['longitude', 'latitude']].values)))
    
    # Get events polygons as dict
    events_poly = {}
    for cluster_id, poly in polygons.items():
        mpl_poly =  mpl_path(poly)
        points_msk = mpl_poly.contains_points(points)
        poly_events_df = open_events_df[points_msk]
        poly_events_ids = list(poly_events_df['id'])
        for event_id in poly_events_ids:
            events_poly[event_id] = cluster_id
    
    # Update events dataframe with events polygons ids
    open_events_df['cluster_id'] = -1
    for event_id, cluster_id in events_poly.items():
        open_events_df['cluster_id'][open_events_df['id']==event_id] = cluster_id

    # Return extended open events
    return open_events_df

def get_clusters_events():
    # Get open events dataframe
    open_events_df = get_open_events()
    # Get events points array
    points = np.array(list(map(tuple, open_events_df[['longitude', 'latitude']].values)))
    # Get clusters polygons events dict
    poly_events = []
    for cluster_id, poly in polygons.items():
        mpl_poly =  mpl_path(poly)
        points_msk = mpl_poly.contains_points(points)
        poly_events_df = open_events_df[points_msk]
        poly_events_ids = list(poly_events_df['id'])
        poly_wb_events_ids = list(poly_events_df['id'][poly_events_df['pop_titulo']=="Bolsão d'água em via"])
        poly_events.append({
            'cluster_id': cluster_id,
            'events': len(poly_events_ids),
            'events_ids': poly_events_ids,
            'events_status': int(bool(len(poly_events_ids))),
            'waterbag_events': len(poly_wb_events_ids),
            'waterbag_events_ids': poly_wb_events_ids,
            'waterbag_events_status': int(bool(len(poly_wb_events_ids))),
        })
    return pd.DataFrame(poly_events)

---
## Pop's (Procedimento Operacional Padrão)

In [60]:
pops = json.loads(requests.get(pops_url).text)['objeto']

pops_df =  pd.DataFrame(pops)

pops_df.to_csv('pops.csv')

pops_df.head()

,titulo,id
0,Abalroamento,24
1,Acidente ambiental,34
2,Acidente com queda de carga,13
3,Acidente com vítima(s),2
4,Acidente com vítima(s) fatal(is),3


---
## Pop's details

In [61]:
pop_details = []
for index, (title, _id) in pops_df.iterrows():
    pop = {'error': ''}
    while 'error' in list(pop.keys()):
        try:
            res = requests.get(pop_url + '?popId=' + str(_id))
            pop = json.loads(res.text)
            pop_df = pd.DataFrame(pop['atividades'])
            pop_df['id'], pop_df['titulo'] = _id, title
            pop_details.append(pop_df)
        except:
            print(f'Error for popId {_id} and title {title}. Retrying...' )
pop_details = pd.concat(pop_details).reset_index(drop=True)

# save pops details dataframe
pop_details.to_csv('pops_actions.csv')

pop_details.head()

,sigla,orgao,acao,id,titulo
0,CET-RIO,Companhia de Engenharia de Tráfego,Organizar o trânsito,24,Abalroamento
1,COMLURB,Companhia de Limpeza Urbana,Fazer limpeza do local,24,Abalroamento
2,Light,Light Serviços de Eletricidade S.A.,Desligar energia,24,Abalroamento
3,Light,Light Serviços de Eletricidade S.A.,Restabelecer fornecimento de energia,24,Abalroamento
4,RioLuz,Companhia Municipal de Energia e Iluminação,Desligar energia,24,Abalroamento


---
## Open Events Example

In [251]:
open_events_df = get_open_events()

open_events_df.to_csv('open_events.csv') # save as csv

open_events_df.head()

Api comando open events: Request attempt (1)


,tipo,pop_id,bairro,latitude,anexos,inicio,titulo,prazo,descricao,informe_id,gravidade,id,longitude,status,pop_titulo
0,SECUNDARIO,25,Vidigal,-22.992024,[AgACAgEAAxkDAAEOU8xi0v9qUn5_qoLI4GtkDrDwjbEAA...,2022-07-16 15:06:18.0,Obra na Via (GEORIO),LONGO,Av. Niemeyer - Alt. Hotel Sheraton - Leblon,77959,BAIXO,77958,-43.233782,ABERTO,Obra em local público
1,SECUNDARIO,25,Lagoa,-22.972271,[AgACAgEAAxkDAAEObI5jIcM6NjZGyb5Urm39wuJPQrWt_...,2022-08-06 08:03:16.0,Obra na Via (Cam 259),LONGO,Av. Borges de Medeiros - Alt. Parque dos Patin...,78966,BAIXO,78965,-43.216449,ABERTO,Obra em local público
2,SECUNDARIO,25,Lagoa,-22.963160,NaN,2022-10-26 20:42:23.0,Asfalto Liso (SECONSERVA),LONGO,Av. Borges de Medeiros - Alt. Clube do Vasco -...,83265,BAIXO,83264,-43.208749,ABERTO,Obra em local público
3,PRIMARIO,7,Centro,-22.903348,NaN,2022-11-04 01:59:47.0,Manifestação,LONGO,Praça Duque de Caxias - Frente Comando Militar...,83773,MEDIO,83772,-43.189722,ABERTO,Manifestação em local público
4,SECUNDARIO,23,Laranjeiras,-22.931563,NaN,2022-11-08 07:54:27.0,Buraco na via,MEDIO,"R. das Laranjeiras, 1 - Laranjeiras",84001,BAIXO,84000,-43.179405,ABERTO,Afundamento de Pista / Buraco na via


---
## Open Events Actions

In [ ]:
# ...

event_actions_url + '?eventId=' + str(eventId)

---
## Points inside polygons function

### Points inside polygon example

In [144]:
polygon = [[0,0], [0,1], [1,1], [1,0]]
points = [(0.1, 0.5), (0.6, 0.3), (2, 0.3)]
poly =  mpl_path(polygon)
print(poly.contains_points(points))

[ True  True False]


### Get open events' points' polygons - Events extented

In [252]:
open_events_df = get_open_events_extended()

# Save and display example extended events dataframe
open_events_df.to_csv('open_events_extended.csv') # save as csv
open_events_df.head()

Api comando open events: Request attempt (1)


<ipython-input-250-b02d7341c2e6>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  open_events_df['cluster_id'][open_events_df['id']==event_id] = cluster_id


,tipo,pop_id,bairro,latitude,anexos,inicio,titulo,prazo,descricao,informe_id,gravidade,id,longitude,status,pop_titulo,cluster_id
0,SECUNDARIO,25,Vidigal,-22.992024,[AgACAgEAAxkDAAEOU8xi0v9qUn5_qoLI4GtkDrDwjbEAA...,2022-07-16 15:06:18.0,Obra na Via (GEORIO),LONGO,Av. Niemeyer - Alt. Hotel Sheraton - Leblon,77959,BAIXO,77958,-43.233782,ABERTO,Obra em local público,-1
1,SECUNDARIO,25,Lagoa,-22.972271,[AgACAgEAAxkDAAEObI5jIcM6NjZGyb5Urm39wuJPQrWt_...,2022-08-06 08:03:16.0,Obra na Via (Cam 259),LONGO,Av. Borges de Medeiros - Alt. Parque dos Patin...,78966,BAIXO,78965,-43.216449,ABERTO,Obra em local público,15
2,SECUNDARIO,25,Lagoa,-22.963160,NaN,2022-10-26 20:42:23.0,Asfalto Liso (SECONSERVA),LONGO,Av. Borges de Medeiros - Alt. Clube do Vasco -...,83265,BAIXO,83264,-43.208749,ABERTO,Obra em local público,-1
3,PRIMARIO,7,Centro,-22.903348,NaN,2022-11-04 01:59:47.0,Manifestação,LONGO,Praça Duque de Caxias - Frente Comando Militar...,83773,MEDIO,83772,-43.189722,ABERTO,Manifestação em local público,-1
4,SECUNDARIO,23,Laranjeiras,-22.931563,NaN,2022-11-08 07:54:27.0,Buraco na via,MEDIO,"R. das Laranjeiras, 1 - Laranjeiras",84001,BAIXO,84000,-43.179405,ABERTO,Afundamento de Pista / Buraco na via,36


### Get polygons open events

In [255]:
poly_events_df = get_clusters_events()

# Save and display example clusters open events dataframe
poly_events_df.to_csv('clusters_open_events.csv') # save as csv
poly_events_df.head()

Api comando open events: Request attempt (1)


,cluster_id,events,events_ids,events_status,waterbag_events,waterbag_events_ids,waterbag_events_status
0,-1,14,"[77958, 78965, 83264, 83772, 84000, 84076, 842...",1,0,[],0
1,0,0,[],0,0,[],0
2,1,0,[],0,0,[],0
3,2,0,[],0,0,[],0
4,3,0,[],0,0,[],0


---
## Basic polygons' events desription

In [256]:
events_now = open_events_df.copy()

#### Events inside polygons

In [257]:
events_inside = (events_now['cluster_id'] != -1).sum()
events_outside = len(events_now) - events_inside

print('Eventos na cidade:', len(events_now))
print('Eventos dentro dos polígonos:', events_inside)
print('Eventos fora dos polígonos', events_outside)

Eventos na cidade: 14
Eventos dentro dos polígonos: 2
Eventos fora dos polígonos 12
